In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
import random
import math
import operator

# Handle Data

In [2]:
with open('iris.data.txt', 'r') as csvfile:
    lines = csv.reader(csvfile)
    row = next(lines)
    for row in lines :
        print (', '.join(row))
        

4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4, 1.5, 0.4, Iris-setosa
5.2, 4.1

In [3]:

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        print(len(dataset))
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
                
            else:
                testSet.append(dataset[x])
        #Converting categorical to numerical so we can use knn
            switcher ={   
            'Iris-setosa':0,
             'Iris-versicolor':1,
            'Iris-virginica':2    
       

    }
            dataset[x][4]=switcher.get(dataset[x][4],"Invalid day ") 
                 

In [4]:
trainingSet=[]

testSet=[]

loadDataset('iris.data.txt', 0.66, trainingSet, testSet)

print ('Train: ' + repr(len(trainingSet)))

print ('Test: ' + repr(len(testSet)) )


150
Train: 96
Test: 54


# Similarity

In [5]:
def euclideanDistance(instance1, instance2, length): 
    tot=0
    for i in range(length):
        tot=tot+math.pow(instance1[i]-instance2[i],2)
    return math.sqrt(tot)

In [6]:
instance1=trainingSet
instance2=testSet

for i in range (len(instance2)):
    distance = euclideanDistance(instance1[i], instance2[i],4)
    print ('Distance: ' + repr(distance))

Distance: 0.5385164807134502
Distance: 0.264575131106459
Distance: 0.2999999999999997
Distance: 0.529150262212918
Distance: 1.1618950038622253
Distance: 0.4242640687119283
Distance: 0.3605551275463991
Distance: 0.479583152331272
Distance: 1.1180339887498947
Distance: 1.3601470508735443
Distance: 1.2247448713915896
Distance: 1.0535653752852738
Distance: 0.5477225575051661
Distance: 0.5477225575051664
Distance: 0.953939201416946
Distance: 0.19999999999999993
Distance: 0.30000000000000004
Distance: 1.3114877048604003
Distance: 1.058300524425836
Distance: 0.3464101615137755
Distance: 3.6359317925395684
Distance: 3.988734135035826
Distance: 4.11703777004778
Distance: 3.023243291566195
Distance: 2.2825424421026654
Distance: 3.209361307176243
Distance: 2.9782545223670858
Distance: 3.854867053479277
Distance: 3.749666651850535
Distance: 3.0708305065568178
Distance: 0.5385164807134506
Distance: 0.31622776601683794
Distance: 0.4358898943540673
Distance: 0.3741657386773946
Distance: 0.45825756949

# Neighbors


In [7]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

# Response

In [8]:
# From the given 5 neighbors we will let the value to vote fot the most appropriate one
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][len(neighbors[0])-1] 
        if response in classVotes:
             classVotes[response]+=1
            
        else: 
            classVotes[response]=1
            
    
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [9]:
# We are selecting the 5 nearest neighbors for each test value
response=[]
k=5
for i in range(len(testSet)):
    neighbors =getNeighbors(trainingSet,testSet[i], k)
    response.append(getResponse(neighbors) )

# Accuracy

In [10]:
def getAccuracy(testSet, predictions):
    correct=0
   
    for i in range(len(predictions)):
        
        if(predictions[i]==testSet[i][4]):
            correct+=1
           
    return (correct/float(len(testSet))) * 100.0


In [11]:
getAccuracy(testSet,response)

98.14814814814815

# Manhattan distance 

In [12]:
def manhattanDistance(instance1, instance2, length): 
    tot=0
    for i in range(length):
        tot=tot+abs(instance1[i]-instance2[i])
    return tot

In [13]:
instance1=trainingSet
instance2=testSet

for i in range (len(instance2)):
    distance = manhattanDistance(instance1[i], instance2[i],4)
    print ('Distance: ' + repr(distance))

Distance: 0.6999999999999993
Distance: 0.5000000000000001
Distance: 0.49999999999999956
Distance: 0.7999999999999997
Distance: 2.1000000000000005
Distance: 0.5999999999999995
Distance: 0.5000000000000002
Distance: 0.9000000000000001
Distance: 2.0999999999999996
Distance: 2.3
Distance: 1.8000000000000007
Distance: 1.9
Distance: 0.9999999999999999
Distance: 1.0000000000000004
Distance: 1.3000000000000005
Distance: 0.3999999999999998
Distance: 0.5000000000000001
Distance: 2.0
Distance: 1.5999999999999996
Distance: 0.6000000000000004
Distance: 6.0
Distance: 6.3
Distance: 6.699999999999999
Distance: 5.3999999999999995
Distance: 3.9
Distance: 5.800000000000001
Distance: 4.699999999999999
Distance: 7.000000000000001
Distance: 6.0
Distance: 5.299999999999999
Distance: 0.9000000000000004
Distance: 0.5999999999999999
Distance: 0.7
Distance: 0.600000000000001
Distance: 0.7000000000000011
Distance: 3.499999999999999
Distance: 2.2999999999999994
Distance: 6.0
Distance: 3.1999999999999993
Distance: 

In [14]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = manhattanDistance(testInstance, trainingSet[x],4)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [15]:
# We are selecting the 5 nearest neighbors for each test value
response=[]
k=5
for i in range(len(testSet)):
    neighbors =getNeighbors(trainingSet,testSet[i], k)
    response.append(getResponse(neighbors) )


In [16]:
getAccuracy(testSet,response)

90.74074074074075

# minkowski distance

In [17]:

from math import *
from decimal import Decimal 

def p_root(value, root): 
      
    root_value = 1 / float(root) 
    return round (Decimal(value) **
             Decimal(root_value), 3) 
  
def minkowski_distance(x, y, p_value): 
      
    # pass the p_root function to calculate 
    # all the value of vector parallely  
    return (p_root(sum(pow(abs(a-b), p_value) 
            for a, b in zip(x, y)), p_value)) 

In [18]:
instance1=trainingSet
instance2=testSet

for i in range (len(instance2)):
    distance=minkowski_distance(instance1[i], instance2[i], 4)

    print ('Distance: ' + repr(distance))

Distance: Decimal('0.503')
Distance: Decimal('0.209')
Distance: Decimal('0.240')
Distance: Decimal('0.501')
Distance: Decimal('0.946')
Distance: Decimal('0.401')
Distance: Decimal('0.314')
Distance: Decimal('0.366')
Distance: Decimal('0.870')
Distance: Decimal('1.082')
Distance: Decimal('1.055')
Distance: Decimal('0.837')
Distance: Decimal('0.434')
Distance: Decimal('0.434')
Distance: Decimal('0.903')
Distance: Decimal('0.141')
Distance: Decimal('0.240')
Distance: Decimal('1.143')
Distance: Decimal('1.001')
Distance: Decimal('0.303')
Distance: Decimal('3.088')
Distance: Decimal('3.544')
Distance: Decimal('3.467')
Distance: Decimal('2.564')
Distance: Decimal('2.045')
Distance: Decimal('2.630')
Distance: Decimal('2.728')
Distance: Decimal('3.192')
Distance: Decimal('3.353')
Distance: Decimal('2.655')
Distance: Decimal('0.433')
Distance: Decimal('0.241')
Distance: Decimal('0.357')
Distance: Decimal('0.315')
Distance: Decimal('0.406')
Distance: Decimal('1.502')
Distance: Decimal('1.344')
D

In [19]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = minkowski_distance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [20]:
response=[]
k=5
for i in range(len(testSet)):
    neighbors =getNeighbors(trainingSet,testSet[i], k)
    response.append(getResponse(neighbors) )

In [21]:
getAccuracy(testSet,response)

100.0